In [1]:
import sys

def cadena(fun, var):
    derivada = diff(fun, n2)*diff(fn2,var)\
             + diff(fun, p2)*diff(fp2,var)\
             + diff(fun, x2)*diff(fx2,var)
    
    return derivada

def cadena2(fun, var):
    derivada = diff(fun, n2, n2)*diff(fn2, r)^2 + diff(fun, n2)*diff(fn2, r, r)\
             + diff(fun, p2, p2)*diff(fp2, r)^2 + diff(fun, p2)*diff(fp2, r, r)\
             + diff(fun, x2, x2)*diff(fx2, r)^2 + diff(fun, x2)*diff(fx2, r, r)

    return derivada

In [2]:
#Definimos variables y funciones
var('eps', latex_name = '\\varepsilon')
var('t')
var('r')
var('th', latex_name = '\\theta')
var('ph', latex_name = '\\varphi')
coords = (t,r,th,ph)

h = function('h')(r,th)
k = function('k')(r,th)
v = function('v')(r,th)
omega = function('omega', latex_name='\\omega')(r)

nu = function('nu', latex_name='\\nu')(r)
lamb = function('lamb', latex_name='\\lambda')(r)

#Definimos las funciones N, V, K y Nphi (ecuación 23 AC)
Ne = e^(nu/2)*(1+1/2*eps^2*h)
Ve = e^(lamb)*(1+eps^2*v)
Ke = r^2*(1+eps^2*k)
Nphie = eps*omega

# <center> Tensor de Einstein </center>

In [3]:
gdd = matrix(SR, 4, 4)

gdd[0,0] = -exp(nu)
gdd[1,1] = exp(lamb)
gdd[2,2] = r^2
gdd[3,3] = r^2*sin(th)^2

guu = 1/gdd

K1 = matrix(SR, 4, 4)
K1[0,3] = -omega*r^2*sin(th)^2
K1[3,0] = -omega*r^2*sin(th)^2

K2 = matrix(SR, 4, 4)
K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
K2[1,1] = 4*exp(lamb)*v
K2[2,2] = 4*k*r^2
K2[3,3] = 4*k*r^2*sin(th)^2

In [4]:
gedd = matrix(SR, 4, 4)
for ii in range(0,4):
    for jj in range(0,4):
        gedd[ii,jj] = gdd[ii,jj] + eps*K1[ii,jj] + 1/2*eps^2*K2[ii,jj]
        
geuu = 1/gedd
    
chris = [[[0 for kk in range(0,4)] for jj in range(0,4)] for ii in range(0,4)]
for ii in range(0,4):
    for jj in range(0,4):
        for kk in range(0,4):
            chris[ii][jj][kk] = 0.5*sum(geuu[ii,dd]*(diff(gedd[dd,jj],coords[kk])
                                                   + diff(gedd[dd,kk],coords[jj])
                                                   - diff(gedd[jj,kk],coords[dd])) for dd in range(0,4))

riem = [[[[0 for ll in range(0,4)] for kk in range(0,4)] for jj in range(0,4)] for ii in range(0,4)]
for ii in range(0,4):
    for jj in range(0,4):
        for kk in range(0,4):
            for ll in range(0,4):
                riem[ii][jj][kk][ll] = (diff(chris[ii][jj][ll],coords[kk])
                                      - diff(chris[ii][jj][kk],coords[ll])
                                      + sum(chris[ii][kk][dd]*chris[dd][ll][jj]
                                      - chris[ii][ll][dd]*chris[dd][kk][jj] for dd in range(0,4)))

ricci = matrix(SR, 4, 4)
for ii in range(0,4):
    for kk in range(0,4):
        ricci[ii,kk] = sum(riem[dd][ii][dd][kk] for dd in range(0,4))

s_curv = sum(sum(ricci[ii,jj]*geuu[ii,jj] for ii in range(0,4)) for jj in range(0,4))

Ge = matrix(SR, 4, 4)
Ge = ricci - 0.5*gedd*s_curv

dGe = matrix(SR, 4, 4)
ddGe = matrix(SR, 4, 4)
for ii in range(0,4):
    for jj in range(0,4):
        dGe[ii,jj] = diff(Ge[ii,jj],eps)
        ddGe[ii,jj] = diff(Ge[ii,jj],eps,eps)

# <center> Tensor de energía-momento </center>

In [5]:
#Funciones dependientes de eps
Omegane = function('Omegane', latex_name='\\Omega_{n\\varepsilon}')(eps)
Omegape = function('Omegape', latex_name='\\Omega_{p\\varepsilon}')(eps)

Nne = function('Nne', latex_name='N_{n\\varepsilon}')(r,th,eps)
Npe = function('Npe', latex_name='N_{p\\varepsilon}')(r,th,eps)

#Tomando eps=0
Nn = function('Nn', latex_name='N_{n}')(r,th)
Np = function('Np', latex_name='N_{p}')(r,th)

sus_func = [eps==0,
            Omegane.subs(eps==0)==0, #Staticity at the background
            Nne.subs(eps==0)==Nn,
            Omegape.subs(eps==0)==0, #Staticity at the background
            Npe.subs(eps==0)==Np]

## <center> Campo $u$ de los neutrones </center>

In [6]:
uevec = [Nne,0,0,Nne*Omegane]
Uevec = [0,0,0,0]

for ii in range(0,4):
    for jj in range(0,4):
        Uevec[ii] += uevec[jj]*gedd[ii,jj]

norm = 0
for ii in range(0,4):
    norm += uevec[ii]*Uevec[ii]

Nne_value = sqrt(solve(norm==-1, Nne^2)[0].rhs())

for ii in range(4):
    uevec[ii] = uevec[ii].subs(Nne==Nne_value)
    Uevec[ii] = Uevec[ii].subs(Nne==Nne_value)

Nn_value = copy(Nne_value)
for ii in range(len(sus_func)):
    Nn_value = Nn_value.subs(sus_func[ii])

sus_func.extend([Nn==Nn_value])

uvec = copy(uevec)
for ii in range(4):
    for jj in range(len(sus_func)):
        uvec[ii] = uvec[ii].subs(sus_func[jj])

Nne_paper = 1/sqrt(Ne^2 - sin(th)^2*Ke*(Nphie-Omegane)^2)

uvec_paper = [Nne_paper,0,0,Omegane*Nne_paper]
for ii in range(4):
    for jj in range(len(sus_func)):
        uvec_paper[ii] = uvec_paper[ii].subs(sus_func[jj])

uvec_check = [x-y for (x,y) in zip(uvec, uvec_paper)]

show(LatexExpr("u \\rightarrow "), bool(uvec_check==[0,0,0,0]))

u \rightarrow  True

In [7]:
var('Omegan1', latex_name="\\Omega^{(1)}_{n}")

sus_func.extend([diff(Omegane,eps,eps).subs(eps==0) == 0, #En nuestro gauge Omega2=0
                 diff(Omegane,eps).subs(eps==0) == Omegan1])

uvec1 = [0,0,0,0]
uvec2 = [0,0,0,0]

for ii in range(4):
    uvec1[ii] = diff(uevec[ii],eps)
    uvec2[ii] = diff(uevec[ii],eps,eps)

for ii in range(4):
    for jj in range(len(sus_func)):
        uvec1[ii] = uvec1[ii].subs(sus_func[jj])
        uvec2[ii] = uvec2[ii].subs(sus_func[jj])

uvec1_paper = [0,0,0,0]
uvec1_paper[0] = 1/2*e^(-3*nu/2)*K1[0,0]
uvec1_paper[3] = e^(-nu/2)*Omegan1

uvec2_paper = [0,0,0,0]
uvec2_paper[0] = e^(-3*nu/2)*(1/2*K2[0,0]+3/4*e^(-nu)*K1[0,0]+2*Omegan1*K1[0,3]+Omegan1^2*r^2*sin(th)^2)
uvec2_paper[3] = e^(-nu/2)*(e^(-nu)*K1[0,0]*Omegan1)  

uvec1_check = [ii-jj for (ii,jj) in zip(uvec1, uvec1_paper)]
uvec2_check = [ii-jj for (ii,jj) in zip(uvec2, uvec2_paper)]

show(LatexExpr("u^{(1)} \\rightarrow"), uvec1_check==[0,0,0,0])
show(LatexExpr("u^{(2)} \\rightarrow"), uvec2_check==[0,0,0,0])

u^{(1)} \rightarrow True

u^{(2)} \rightarrow True

## <center> Campo $v$ de los protones </center>

In [9]:
vevec = [Npe,0,0,Npe*Omegape]
Vevec = [0,0,0,0]

for ii in range(4):
    for jj in range(4):
        Vevec[ii] += vevec[jj]*gedd[ii,jj]

norm = 0
for ii in range(4):
    norm += vevec[ii]*Vevec[ii]

Npe_value = sqrt(solve(norm==-1, Npe^2)[0].rhs())
for ii in range(4):
    vevec[ii] = vevec[ii].subs(Npe==Npe_value)
    Vevec[ii] = Vevec[ii].subs(Npe==Npe_value)

Np_value = copy(Npe_value)
for ii in range(len(sus_func)):
    Np_value = Np_value.subs(sus_func[ii])

sus_func.extend([Np==Np_value])

vvec = copy(vevec)
for ii in range(4):
    for jj in range(len(sus_func)):
        vvec[ii] = vvec[ii].subs(sus_func[jj])

Npe_paper = 1/sqrt(Ne^2 - sin(th)^2*Ke*(Nphie-Omegape)^2)

vvec_paper = [Npe_paper,0,0,Omegape*Npe_paper]

for ii in range(4):
    for jj in range(len(sus_func)):
        vvec_paper[ii] = vvec_paper[ii].subs(sus_func[jj])

vvec_check = [x-y for (x,y) in zip(vvec, vvec_paper)]

show(LatexExpr("v \\rightarrow "), bool(vvec_check==[0,0,0,0]))

v \rightarrow  True

In [10]:
var('Omegap1', latex_name="\\Omega^{(1)}_{p}")

sus_func.extend([diff(Omegape,eps,eps).subs(eps==0) == 0, #En nuestro gauge Omega2=0
                 diff(Omegape,eps).subs(eps==0) == Omegap1])

vvec1 = [0,0,0,0]
vvec2 = [0,0,0,0]

for ii in range(4):
    vvec1[ii] = diff(vevec[ii],eps)
    vvec2[ii] = diff(vevec[ii],eps,eps)

for ii in range(4):
    for jj in range(len(sus_func)):
        vvec1[ii] = vvec1[ii].subs(sus_func[jj])
        vvec2[ii] = vvec2[ii].subs(sus_func[jj])

vvec1_paper = [0,0,0,0]
vvec1_paper[0] = 1/2*e^(-3*nu/2)*K1[0,0]
vvec1_paper[3] = e^(-nu/2)*Omegap1

vvec2_paper = [0,0,0,0]
vvec2_paper[0] = e^(-3*nu/2)*(1/2*K2[0,0]+3/4*e^(-nu)*K1[0,0]+2*Omegap1*K1[0,3]+Omegap1^2*r^2*sin(th)^2)
vvec2_paper[3] = e^(-nu/2)*(e^(-nu)*K1[0,0]*Omegap1)  

vvec1_check = [ii-jj for (ii,jj) in zip(vvec1, vvec1_paper)]
vvec2_check = [ii-jj for (ii,jj) in zip(vvec2, vvec2_paper)]

show(LatexExpr("v^{(1)} \\rightarrow"), vvec1_check==[0,0,0,0])
show(LatexExpr("v^{(2)} \\rightarrow"), vvec2_check==[0,0,0,0])

v^{(1)} \rightarrow True

v^{(2)} \rightarrow True

## <center> Ecuación de Einstein </center>

In [11]:
n = function('n')(r,th,eps)
p = function('p')(r,th,eps)

n0 = function('n0', latex_name="n_0")(r)
p0 = function('p0', latex_name="p_0")(r)

eta = function('eta', latex_name="\\eta")(r,th)
Phi = function('Phi', latex_name="\\Phi")(r,th)

n = n0 + 1/2*eps^2*eta
p = p0 + 1/2*eps^2*Phi

#Defino esto para poder hacer derivadas "simbólicas"
var('n2', latex_name="n^2")
var('p2', latex_name="p^2")
var('x2', latex_name="x^2")

Lambdae = function('Lambdae', latex_name='\\Lambda_{\\varepsilon}')(n2,p2,x2,eps,r)
Psie = function('Psie', latex_name='\\Psi_{\\varepsilon}')(n2,p2,x2,eps,r)

nevec = [n*ii for ii in uevec] #eq 13 AC
pevec = [p*ii for ii in vevec] #eq 13 AC

ned = [0,0,0,0]
ped = [0,0,0,0]
for ii in range(4):
    for jj in range(4):
        ned[ii] += nevec[jj]*gedd[ii,jj]
        ped[ii] += pevec[jj]*gedd[ii,jj]

nd = copy(ned)
pd = copy(ped)
for ii in range(4):
    for jj in range(len(sus_func)):
        nd[ii] = nd[ii].subs(sus_func[jj])
        pd[ii] = pd[ii].subs(sus_func[jj])

#Para poder hacer la regla de la cadena necesitamos una expresión explícita para n2, p2 y x2
fn2, fp2, fx2 = 0, 0, 0
for ii in range(4):
    for jj in range(4):
        fn2 += -nevec[ii]*nevec[jj]*gedd[ii,jj]
        fp2 += -pevec[ii]*pevec[jj]*gedd[ii,jj]
        fx2 += -nevec[ii]*pevec[jj]*gedd[ii,jj]

A = -diff(Lambdae, x2)
B = -2*diff(Lambdae, n2)
C = -2*diff(Lambdae, p2)

mued = [B*ii for ii in ned] + [A*jj for jj in ped] #eq 2 C
chied = [C*ii for ii in ped] + [A*jj for jj in ned] #eq 2 C

muevec = [0,0,0,0]
chievec = [0,0,0,0]
for ii in range(4):
    for jj in range(4):
        muevec[ii] += mued[jj]*gedd[ii,jj]
        chievec[ii] += chied[jj]*gedd[ii,jj]

Psie = Lambdae - sum([ii*jj for (ii,jj) in zip(nevec, mued)]) - sum([ii*jj for (ii,jj) in zip(pevec, chied)])

LambdaePsie = Psie - Lambdae

In [12]:
dirac = diagonal_matrix([1, 1, 1, 1])

Teud = matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        Teud[ii,jj] = Psie*dirac[ii,jj] + pevec[ii]*chied[jj] + nevec[ii]*mued[jj]

Te, dTe, ddTe = matrix(SR, 4, 4), matrix(SR, 4, 4), matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        for kk in range(4):
            Te[ii,jj] += Teud[kk,jj]*gedd[kk,ii]
        
        dTe[ii,jj] = diff(Te[ii,jj],eps)
        ddTe[ii,jj] = diff(Te[ii,jj],eps,eps)

kappa = 8*pi

In [13]:
dLambdae_dr = cadena(Lambdae, r)
ddLambdae_dn2_dr = diff(cadena(Lambdae, r), n2)
ddLambdae_dp2_dr = diff(cadena(Lambdae, r), p2)
ddLambdae_dx2_dr = diff(cadena(Lambdae, r), x2)
for ii in range(len(sus_func)):
    dLambdae_dr = dLambdae_dr.subs(sus_func[ii])
    ddLambdae_dn2_dr = ddLambdae_dn2_dr.subs(sus_func[ii])
    ddLambdae_dp2_dr = ddLambdae_dp2_dr.subs(sus_func[ii])
    ddLambdae_dx2_dr = ddLambdae_dx2_dr.subs(sus_func[ii])

sus_func.extend([diff(Lambdae, r).subs(eps==0) == dLambdae_dr,
                 diff(Lambdae, n2, r).subs(eps==0) == ddLambdae_dn2_dr,
                 diff(Lambdae, p2, r).subs(eps==0) == ddLambdae_dp2_dr,
                 diff(Lambdae, x2, r).subs(eps==0) == ddLambdae_dx2_dr])

In [14]:
mue = B*n + A*p #eq 22 C
chie = A*n + C*p #eq 22 C

#Volvemos a definir mu_d y chi_d porque los que habíamos definido antes eran de AC y este es de C
mued = [mue*ii for ii in Uevec] #eq 17 C
chied = [chie*ii for ii in Uevec] #eq 17 C

muinf = -mued[0] #eq 20
chiinf = -chied[0] #eq 20
for ii in range(len(sus_func)):
    muinf = muinf.subs(sus_func[ii])
    chiinf = chiinf.subs(sus_func[ii])

eq_mu = diff(muinf,r)
eq_chi = diff(chiinf,r)
for ii in range(len(sus_func)):
    eq_mu = eq_mu.subs(sus_func[ii])
    eq_chi = eq_chi.subs(sus_func[ii])

A0, B0, C0 = A, B, C

A00 = A + 2*diff(B,p2)*n*p + 2*diff(A,n2)*n^2 + 2*diff(A,p2)*p^2 + diff(A,x2)*p*n
B00 = B + 2*diff(B,n2)*n^2 + 4*diff(A,n2)*n*p + diff(A,x2)*p^2
C00 = C + 2*diff(C,p2)*p^2 + 4*diff(A,p2)*n*p + diff(A,x2)*n^2

A000, B000, C000 = A00, B00, C00
for ii in range(len(sus_func)):
    A0 = A0.subs(sus_func[ii])
    B0 = B0.subs(sus_func[ii])
    C0 = C0.subs(sus_func[ii])
    A000 = A000.subs(sus_func[ii])
    B000 = B000.subs(sus_func[ii])
    C000 = C000.subs(sus_func[ii])

eq26_1 = A000*diff(p0,r) + B000*diff(n0,r) + 1/2*(B0*n0 + A0*p0)*diff(nu,r)
eq26_2 = C000*diff(p0,r) + A000*diff(n0,r) + 1/2*(A0*n0 + C0*p0)*diff(nu,r)
for ii in range(len(sus_func)):
    eq26_1 = eq26_1.subs(sus_func[ii])
    eq26_2 = eq26_2.subs(sus_func[ii])

eq_mu = eq_mu*e^(-nu/2)
eq_mu_check = eq_mu - eq26_1
eq_chi = eq_chi*e^(-nu/2)
eq_chi_check = eq_chi - eq26_2

show(LatexExpr("\\text{Eq 26.1} \\rightarrow"), bool(eq_mu_check==0))
show(LatexExpr("\\text{Eq 26.2} \\rightarrow"), bool(eq_chi_check==0))

\text{Eq 26.1} \rightarrow True

\text{Eq 26.2} \rightarrow True

In [15]:
dp0_value = solve(eq26_1, diff(p0,r))[0].rhs()
for ii in range(len(sus_func)):
    dp0_value = dp0_value.subs(sus_func[ii])
sus_func.extend([diff(p,r).subs(eps==0)==dp0_value])

dn0_value = solve(eq26_2, diff(n0,r))[0].rhs()
for ii in range(len(sus_func)):
    dn0_value = dn0_value.subs(sus_func[ii])
sus_func.extend([diff(n,r).subs(eps==0)==dn0_value])

# <center> $G_{\mu\nu} = \kappa T_{\mu\nu}$ </center>

In [16]:
Lambda0 = function('Lambda0', latex_name='\\Lambda_0')(n2,p2,x2)
Psi0 = function('Psi0', latex_name="\\Psi_0")(n2,p2,x2)

sus_func.extend([Lambdae.subs(eps==0) == Lambda0])

Psi0 = copy(Psie)
for ii in range(len(sus_func)):
    Psi0 = Psi0.subs(sus_func[ii])

G = copy(Ge)
T = copy(Te)
for ii in range(len(sus_func)):
    G = G.subs(sus_func[ii])
    T = T.subs(sus_func[ii])

## <center> $G_{00} = \kappa T_{00}$ </center>

In [17]:
dlamb_value = solve(G[0,0] - kappa*T[0,0] == 0, diff(lamb, r))[0].rhs().simplify_full()
dlamb_paper = (1-e^(lamb))/r - 8*pi*r*e^(lamb)*Lambda0
dlamb_check = dlamb_value - dlamb_paper
show(LatexExpr("\\lambda' \\rightarrow"), bool(dlamb_check==0))

for ii in range(len(sus_func)):
    dlamb_value = dlamb_value.subs(sus_func[ii])

sus_func.extend([diff(lamb,r)==dlamb_value])

\lambda' \rightarrow True

## <center> $G_{11} = \kappa T_{11}$ </center>


In [18]:
dnu_value_11 = solve(G[1,1] - kappa*T[1,1] == 0, diff(nu, r))[0].rhs().simplify_full()
dnu_paper = -(1-e^(lamb))/r + 8*pi*r*e^(lamb)*Psi0
dnu_check_11 = dnu_value_11 - dnu_paper
show(LatexExpr("\\nu' \\rightarrow"), bool(dnu_check_11==0))

for ii in range(len(sus_func)):
    dnu_value_11 = dnu_value_11.subs(sus_func[ii])

sus_func.extend([diff(nu,r)==dnu_value_11])

\nu' \rightarrow True

In [19]:
#dnu_value_11 = solve(G[1,1] - kappa*T[1,1] == 0, diff(nu, r))[0].rhs().simplify_full()
#ddnu_value_11 = diff(dnu_value_11,r).simplify_full()
#show(ddnu_value_11)
#for ii in range(len(sus_func)):
#    ddnu_value_11 = ddnu_value_11.subs(sus_func[ii])
#print(ddnu_value_11.simplify_full())
#dnu_paper = -(1-e^(lamb))/r + 8*pi*r*e^(lamb)*Psi0
#dnu_check_11 = dnu_value_11 - dnu_paper
#show(LatexExpr("\\nu' \\rightarrow"), bool(dnu_check_11==0))
#
#for ii in range(len(sus_func)):
#    dnu_value_11 = dnu_value_11.subs(sus_func[ii])

## <center> $G_{11} = \kappa T_{11}$ y $G_{22} = \kappa T_{22}$ </center>


In [20]:
dnu_value_22 = solve(G[2,2] - kappa*T[2,2] == 0, diff(nu, r))[0].rhs().simplify_full()

#Despejamos la raiz cuadrada de la ecuación
dEq_1122 = -(dnu_value_11 - dnu_value_22 == 0)*e^(nu)*2*r
dEq_1122 = dEq_1122 + sqrt(-256*pi*r^2*e^(lamb + 2*nu)*n0^2*diff(Lambdae,n2).subs(eps==0)\
                           -256*pi*r^2*e^(lamb + 2*nu)*p0^2*diff(Lambdae,p2).subs(eps==0)\
                           +r^2*e^(2*nu)*diff(lamb, r)^2\
                           -8*r^2*e^(2*nu)*diff(nu, r, r)\
                           +4*r*e^(2*nu)*diff(lamb, r)\
                           +4*(32*pi*r^2*Lambda0*e^lamb + 1)*e^(2*nu))

ddnu_value = solve(dEq_1122^2, diff(nu,r,r))[0].rhs()
ddnu_paper = 2*r*diff(nu,r,r) + diff(nu,r)*(r*diff(nu,r)-2)-diff(lamb,r)*(2+r*diff(nu,r))+4/r*(e^(lamb)-1) == 0
ddnu_paper = solve(ddnu_paper, diff(nu,r,r))[0].rhs()
ddnu_check = ddnu_value - ddnu_paper
for ii in range(len(sus_func)):
    ddnu_value = ddnu_value.subs(sus_func[ii])
    ddnu_check = ddnu_check.subs(sus_func[ii])
show(LatexExpr("\\nu'' \\rightarrow"), bool(ddnu_check==0))

sus_func.extend([diff(nu,r,r)==ddnu_value])

\nu'' \rightarrow True

# <center> $G^{\prime}_{\mu\nu} = \kappa T^{\prime}_{\mu\nu}$ </center>

In [21]:
dLambdaePsie = diff(LambdaePsie, eps)
for ii in range(len(sus_func)):
    dLambdaePsie = dLambdaePsie.subs(sus_func[ii])

In [22]:
j = e^(-(lamb+nu)/2)
M = r*(1-e^(-lamb))/2

Lambda1 = function('Lambda1', latex_name='\\Lambda^{(1)}')(n2,p2,x2,r) ##
sus_func.extend([diff(Lambdae,eps).subs(eps==0) == Lambda1])
sus_func.extend([diff(Lambda1,r).subs(eps==0) == cadena(Lambda1, r)]) ##

dG = copy(dGe)
dT = copy(dTe)
for ii in range(0,len(sus_func)):
    dG = dG.subs(sus_func[ii])
    dT = dT.subs(sus_func[ii])

In [23]:
dEq_00 = dG[0,0] - kappa*dT[0,0] == 0

Lambda1_value = solve(dEq_00,Lambda1)[0].rhs().simplify_full()

Lambda1_paper = 0
Lambda1_check = Lambda1_value - Lambda1_paper

show(LatexExpr("\\Lambda^{(1)} = "), Lambda1_value, LatexExpr("\\rightarrow"), bool(Lambda1_check==0))

dG = dG.subs(Lambda1==Lambda1_value)
dT = dT.subs(Lambda1==Lambda1_value)
sus_func.extend([Lambda1==Lambda1_value])

\Lambda^{(1)} =  0 \rightarrow True

In [27]:
dEq_11 = dG[1,1] - kappa*dT[1,1] == 0

D03Lambdae = solve(dEq_11, diff(Lambdae, n2,eps).subs(eps==0))[0].rhs()
sus_func.extend([diff(Lambdae, n2,eps).subs(eps==0) == D03Lambdae])

Psi1_value = Lambda1 + dLambdaePsie
for ii in range(len(sus_func)):
    Psi1_value = Psi1_value.subs(sus_func[ii])

Psi1_paper = 0
Psi1_check = Psi1_value - Psi1_paper

show(LatexExpr("\\Psi^{(1)} = "), Psi1_value, LatexExpr("\\rightarrow"), bool(Psi1_check==0))

#Como en nuestras ecuaciones no tenemos Psi1, tenemos que sustituir la diferencia de Psi1 y Lambda1
Psi1_subs = dLambdaePsie == 0
Psi1_subs = solve(Psi1_subs, diff(Lambdae, n2, eps).subs(eps==0))[0].rhs()

dG = dG.subs(diff(Lambdae, n2, eps).subs(eps==0)==Psi1_subs)
dT = dT.subs(diff(Lambdae, n2, eps).subs(eps==0)==Psi1_subs)
sus_func.extend([diff(Lambdae, n2, eps).subs(eps==0)==Psi1_subs])

# ESTO NO ES 0=0?
show(Psi1_subs)
show(diff(Lambdae, n2, eps).subs(eps==0))

\Psi^{(1)} =  0 \rightarrow True

-p0(r)^2*D[1, 3](Lambdae)(n2, p2, x2, 0, r)/n0(r)^2

D[0, 3](Lambdae)(n2, p2, x2, 0, r)

In [25]:
dEq_03 = dG[0,3] - kappa*dT[0,3] == 0

ddomega_value = solve(dEq_03, diff(omega,r,r))[0].rhs().simplify_full()

Ln = (omega - Omegan1)*eps
chi0 = C*p + A*n

ddomega_paper = 1/r^4*diff(r^4*e^(-(lamb+nu)/2)*diff(Ln,r),r)\
                -16*pi*e^((lamb-nu)/2)*(Psi0-Lambda0)*Ln\
                -16*pi*e^((lamb-nu)/2)*chi0*p0*(Omegan1-Omegap1)*eps == 0

ddomega_paper = solve(ddomega_paper, diff(omega,r,r))[0].rhs().simplify_full()

for ii in range(len(sus_func)):
    ddomega_value = ddomega_value.subs(sus_func[ii])
    ddomega_paper = ddomega_paper.subs(sus_func[ii])

ddomega_check = ddomega_value - ddomega_paper
for ii in range(len(sus_func)):
    ddomega_value = ddomega_value.subs(sus_func[ii])
    ddomega_paper = ddomega_paper.subs(sus_func[ii])
    ddomega_check = ddomega_check.subs(sus_func[ii])

show(ddomega_value.simplify_full())
show(ddomega_paper.simplify_full())
show(bool(ddomega_check==0))

show(LatexExpr("\\omega'' = "), ddomega_check.simplify_full())

#sus_func.extend([diff(omega,r,r)==ddomega_value])

4*(8*pi*Omegan1*r*e^lamb(r)*n0(r)^2*D[0](Lambdae)(n2, p2, x2, 0, r) - 8*pi*r*e^lamb(r)*n0(r)^2*omega(r)*D[0](Lambdae)(n2, p2, x2, 0, r) + 8*(pi*Omegap1*r*e^lamb(r)*D[1](Lambdae)(n2, p2, x2, 0, r) - pi*r*e^lamb(r)*omega(r)*D[1](Lambdae)(n2, p2, x2, 0, r))*p0(r)^2 - (2*pi*r^2*e^lamb(r)*n0(r)^2*D[0](Lambdae)(n2, p2, x2, 0, r) + 2*pi*r^2*e^lamb(r)*p0(r)^2*D[1](Lambdae)(n2, p2, x2, 0, r) + 1)*diff(omega(r), r))/r

4*(8*pi*Omegan1*r*e^lamb(r)*n0(r)^2*D[0](Lambdae)(n2, p2, x2, 0, r) - 8*pi*r*e^lamb(r)*n0(r)^2*omega(r)*D[0](Lambdae)(n2, p2, x2, 0, r) - 4*(pi*Omegan1 - pi*Omegap1)*r*e^lamb(r)*n0(r)*p0(r)*D[2](Lambdae)(n2, p2, x2, 0, r) + 8*(pi*Omegap1*r*e^lamb(r)*D[1](Lambdae)(n2, p2, x2, 0, r) - pi*r*e^lamb(r)*omega(r)*D[1](Lambdae)(n2, p2, x2, 0, r))*p0(r)^2 - (2*pi*r^2*e^lamb(r)*n0(r)^2*D[0](Lambdae)(n2, p2, x2, 0, r) + 2*pi*r^2*e^lamb(r)*p0(r)^2*D[1](Lambdae)(n2, p2, x2, 0, r) + 1)*diff(omega(r), r))/r

False

\omega'' =  16*(pi*Omegan1 - pi*Omegap1)*e^lamb(r)*n0(r)*p0(r)*D[2](Lambdae)(n2, p2, x2, 0, r)

# <center> $G^{\prime\prime}_{\mu\nu} = \kappa T^{\prime\prime}_{\mu\nu}$ </center>

In [ ]:
ddLambdaePsie = diff(LambdaePsie, eps, eps)
for ii in range(len(sus_func)):
    ddLambdaePsie = ddLambdaePsie.subs(sus_func[ii])

In [ ]:
Lambda2 = function('Lambda2', latex_name='\\Lambda^{(2)}')(n2,p2,x2)

sus_func.extend([diff(Lambdae,eps,eps).subs(eps==0) == Lambda2])

ddG = copy(ddGe)
ddT = copy(ddTe)
for ii in range(len(sus_func)):
    ddG = ddG.subs(sus_func[ii])
    ddT = ddT.subs(sus_func[ii])

In [ ]:
sys.exit()

## <center> Descomposición en $P_l(\cos(\theta))$ </center>

In [ ]:
# FALTA ECUACIÓN 31 DE ETA Y PHI

In [ ]:
h0 = function('h0', latex_name='h_0')(r)
v0 = function('v0', latex_name='v_0')(r)

h2 = function('h2', latex_name='h_2')(r)
k2 = function('k2', latex_name='k_2')(r)
v2 = function('v2', latex_name='v_2')(r)

Lambda20 = function('Lambda20', latex_name='\\Lambda_{0}^{(2)}')(n2,p2,x2)
Lambda22 = function('Lambda22', latex_name='\\Lambda_{2}^{(2)}')(n2,p2,x2)

LP1 = function('LP1', latex_name='LP_1')(th)
LP2 = function('LP2', latex_name='LP_2')(th)

lp2 = (3*cos(th)^2-1)/2

In [ ]:
sus_descomp = [v==v0+v2*lp2,
               diff(v,r)==diff(v0+v2*lp2,r),
               diff(v,r,r)==diff(v0+v2*lp2,r,r),
               diff(v,th)==diff(v0+v2*lp2,th),
               diff(v,th,th)==diff(v0+v2*lp2,th,th),
               diff(v,th,r)==diff(v0+v2*lp2,th,r),
               diff(v,r,th)==diff(v0+v2*lp2,r,th),
               k==k2*lp2,
               diff(k,r)==diff(k2*lp2,r),
               diff(k,r,r)==diff(k2*lp2,r,r),
               diff(k,th)==diff(k2*lp2,th),
               diff(k,th,th)==diff(k2*lp2,th,th),
               diff(k,th,r)==diff(k2*lp2,th,r),
               diff(k,r,th)==diff(k2*lp2,r,th),
               h==h0+h2*lp2,
               diff(h,r)==diff(h0+h2*lp2,r),
               diff(h,r,r)==diff(h0+h2*lp2,r,r),
               diff(h,th)==diff(h0+h2*lp2,th),
               diff(h,th,th)==diff(h0+h2*lp2,th,th),
               diff(h,th,r)==diff(h0+h2*lp2,th,r),
               diff(h,r,th)==diff(h0+h2*lp2,r,th),
               Lambda2==Lambda20+Lambda22*lp2,
               diff(Lambda2,r)==diff(Lambda20+Lambda22*lp2,r),
               diff(Lambda2,r,r)==diff(Lambda20+Lambda22*lp2,r,r),
               diff(Lambda2,th)==diff(Lambda20+Lambda22*lp2,th),
               diff(Lambda2,th,th)==diff(Lambda20+Lambda22*lp2,th,th),
               diff(Lambda2,th,r)==diff(Lambda20+Lambda22*lp2,th,r),
               diff(Lambda2,r,th)==diff(Lambda20+Lambda22*lp2,r,th),
               Psi2==Psi20+Psi22*lp2,
               diff(Psi2,r)==diff(Psi20+Psi22*lp2,r),
               diff(Psi2,r,r)==diff(Psi20+Psi22*lp2,r,r),
               diff(Psi2,th)==diff(Psi20+Psi22*lp2,th),
               diff(Psi2,th,th)==diff(Psi20+Psi22*lp2,th,th),
               diff(Psi2,th,r)==diff(Psi20+Psi22*lp2,th,r),
               diff(Psi2,r,th)==diff(Psi20+Psi22*lp2,r,th)]

for ii in range(0,len(sus_descomp)):
    ddG = ddG.subs(sus_descomp[ii])
    ddT = ddT.subs(sus_descomp[ii])

ddG = ddG.simplify_full()
ddT = ddT.simplify_full()

In [ ]:
sus_legendre = [sin(th)^2 == (2/3)*(1-LP2),
                cos(th)^2 == (2*LP2+1)/3,
                cos(th) == LP1]

### <center> $\boxed{l=0}$ </center>

### <center> $G^{\prime\prime}_{00} = \kappa T^{\prime\prime}_{00}$ </center>

In [ ]:
ddEq_00 = ddG[0,0] - kappa*ddT[0,0] == 0
show(ddEq_00)
Lambda20_value_00 = solve(ddEq_00,Lambda20)[0].rhs().simplify_full()

show(Lambda20_value_00)

In [ ]:
ddEq_00 = ddG[0,0] - kappa*ddT[0,0] == 0
Lambda20_value_00 = solve(ddEq_00,Lambda20)[0].rhs().simplify_full()

for ii in range(0,len(sus_legendre)):
    E20_value_00 = E20_value_00.subs(sus_legendre[ii])

E20_value_00 = E20_value_00.subs(LP1==0).subs(LP2==0).simplify_full()

ddG = ddG.subs(E20==E20_value_00)
ddT = ddT.subs(E20==E20_value_00)
sus_func.extend([E20==E20_value_00])
    
E20_paper = (4/r^2*diff(r*e^(-lamb)*v0,r)\
             +8/3*r*j*diff(j,r)*(omega-Omega1)^2\
             -1/3*j^2*r^2*diff(omega,r)^2)/(8*pi)

E20_check = E20_value_00 - E20_paper

show(LatexExpr("E^{(2)}_0 \\rightarrow"), bool(E20_check==0))

### <center> $G^{\prime\prime}_{11} = \kappa T^{\prime\prime}_{11}$ </center>

In [ ]:
ddEq_11 = ddG[1,1] - kappa*ddT[1,1] == 0
P20_value = solve(ddEq_11,P20)[0].rhs().simplify_full()

for ii in range(0,len(sus_legendre)):
    P20_value = P20_value.subs(sus_legendre[ii])
    
P20_value = P20_value.subs(LP1==0).subs(LP2==0).simplify_full()
dP20_value = diff(P20_value,r)
for ii in range(0,len(sus_func)):
    dP20_value = dP20_value.subs(sus_func[ii])
    
ddG = ddG.subs(diff(P20,r)==dP20_value).subs(P20==P20_value)
ddT = ddT.subs(diff(P20,r)==dP20_value).subs(P20==P20_value)
sus_func.extend([diff(P20,r)==dP20_value,
                 P20==P20_value])

P20_paper = (4/r^2*(r*e^(-lamb)*diff(h0,r)-v0*(8*pi*r^2*P+1))\
             +1/3*r^2*j^2*diff(omega,r)^2)/(8*pi)

for ii in range(0,len(sus_func)):
    P20_paper = P20_paper.subs(sus_func[ii])

P20_check = P20_value - P20_paper

show(LatexExpr("P^{(2)}_0 \\rightarrow"), bool(P20_check==0))

### <center> $G^{\prime\prime}_{22} = \kappa T^{\prime\prime}_{22}$    y    $G^{\prime\prime}_{33} = \kappa T^{\prime\prime}_{33}$ </center>

In [ ]:
ddEq_22 = ddG[2,2] - kappa*ddT[2,2] == 0
ddh0_value_22 = solve(ddEq_22,diff(h0,r,r))[0].rhs().simplify_full()

ddEq_33 = ddG[3,3] - kappa*ddT[3,3] == 0
ddh0_value_33 = solve(ddEq_33,diff(h0,r,r))[0].rhs().simplify_full()

for ii in range(0,len(sus_legendre)):
    ddh0_value_22 = ddh0_value_22.subs(sus_legendre[ii])
    ddh0_value_33 = ddh0_value_33.subs(sus_legendre[ii])

ddh0_value_22 = ddh0_value_22.subs(LP1==0).subs(LP2==0).simplify_full()
ddh0_value_33 = ddh0_value_33.subs(LP1==0).subs(LP2==0).simplify_full()

ddG = ddG.subs(diff(h0,r,r)==ddh0_value_22)
ddT = ddT.subs(diff(h0,r,r)==ddh0_value_22)
sus_func.extend([diff(h0,r,r)==ddh0_value_22])

v2h2 = ddh0_value_22 - ddh0_value_33 == 0
v2_value = solve(v2h2,v2)[0].rhs().simplify_full()

v2_paper = 1/6*r^4*j^2*diff(omega,r)^2\
          -1/3*r^3*diff(j^2,r)*(omega-Omega1)^2\
          -h2

dv2_value = diff(v2_value,r)
ddv2_value = diff(v2_value,r,r)
for ii in range(0,len(sus_func)):
    dv2_value = dv2_value.subs(sus_func[ii])
    ddv2_value = ddv2_value.subs(sus_func[ii])
    v2_paper = v2_paper.subs(sus_func[ii])

ddG = ddG.subs(diff(v2,r,r)==ddv2_value).subs(diff(v2,r)==dv2_value).subs(v2==v2_value)
ddT = ddT.subs(diff(v2,r,r)==ddv2_value).subs(diff(v2,r)==dv2_value).subs(v2==v2_value)
sus_func.extend([diff(v2,r,r)==ddv2_value,
                 diff(v2,r)==dv2_value,
                 v2==v2_value])

v2_check = v2_value - v2_paper

show(LatexExpr("v_2 \\rightarrow"), bool(v2_check==0))

### <center> Barotropic equation of state </center>

In [ ]:
EOS = E2*diff(P,r) - P2*diff(E,r) == 0

for ii in range(0,len(sus_descomp)):
    EOS = EOS.subs(sus_descomp[ii])
EOS = EOS.simplify_full()

for ii in range(0,len(sus_legendre)):
    EOS = EOS.subs(sus_legendre[ii])

EOS = EOS.subs(LP1==0).subs(LP2==0)

PT0 = P20/(2*(E+P))

E20_value_EOS = solve(EOS, E20)[0].rhs()

dv0_value = E20_value_EOS - E20_value_00 == 0
dv0_value = solve(dv0_value, diff(v0,r))[0].rhs()

dv0_paper = - diff(r*e^(-lamb)*v0,r)\
            + 4*pi*r^2*diff(E,r)/diff(P,r)*(E+P)*PT0\
            + 1/12*j^2*r^4*diff(omega,r)^2\
            - 2/3*r^3*j*diff(j,r)*(omega-Omega1)^2  == 0

dv0_paper = solve(dv0_paper, diff(v0,r))[0].rhs()

ddv0_value = diff(dv0_value,r)
for ii in range(0,len(sus_func)):
    dv0_value = dv0_value.subs(sus_func[ii])
    ddv0_value = ddv0_value.subs(sus_func[ii])
    dv0_paper = dv0_paper.subs(sus_func[ii])
    
ddG = ddG.subs(diff(v0,r,r)==diff(dv0_value,r)).subs(diff(v0,r)==dv0_value)
ddT = ddT.subs(diff(v0,r,r)==diff(dv0_value,r)).subs(diff(v0,r)==dv0_value)
sus_func.extend([diff(v0,r,r)==diff(dv0_value,r),
                 diff(v0,r)==dv0_value])

dv0_check = dv0_value - dv0_paper
show(LatexExpr("v_{0}' \\rightarrow"), bool(dv0_check==0))

### <center> Ecuación 62 </center>

In [ ]:
eq_62 = diff(PT0, r)\
        + 4*pi*(E+P)*r^2*PT0/(r-2*M)\
        + (r*e^(-lamb)*v0)*r^2/(r-2*M)^2*(8*pi*P+1/r^2)\
        - r^4*j^2/(12*(r-2*M))*diff(omega,r)^2\
        - 1/3*diff((r^3*j^2*(omega-Omega1)^2)/(r-2*M),r)

for ii in range(0,len(sus_func)):
    eq_62 = eq_62.subs(sus_func[ii])

show(LatexExpr("\\text{Ecuación (62) }\\rightarrow"), bool(eq_62==0))

### <center> Ecuación 64 </center>

In [ ]:
eq_64 = PT0+h0-1/3*r^2*e^(-nu)*(omega-Omega1)^2
deq_64 = diff(eq_64, r)
for ii in range(0,len(sus_func)):
    deq_64 = deq_64.subs(sus_func[ii])

show(LatexExpr("\\text{Ecuación (64) }\\rightarrow"), bool(deq_64==0))

### <center> $\boxed{l=2}$ </center>

### <center> $G^{\prime\prime}_{22} = \kappa T^{\prime\prime}_{22}$    y    $G^{\prime\prime}_{33} = \kappa T^{\prime\prime}_{33}$ </center>

In [ ]:
ddEq_22 = (ddG[2,2]-kappa*ddT[2,2] == 0).simplify_full()
ddEq_33 = ((ddG[3,3]-kappa*ddT[3,3] == 0)/sin(th)^2).simplify_full()

for ii in range(0,len(sus_legendre)):
    ddEq_22 = ddEq_22.subs(sus_legendre[ii])
    ddEq_33 = ddEq_33.subs(sus_legendre[ii])

ddEq_22 = (ddEq_22.subs(LP1==0) - ddEq_22.subs(LP1==0).subs(LP2==0))/LP2
P22_value_22 = solve(ddEq_22, P22)[0].rhs().simplify_full()

ddEq_33 = (ddEq_33.subs(LP1==0) - ddEq_33.subs(LP1==0).subs(LP2==0))/LP2
P22_value_33 = solve(ddEq_33, P22)[0].rhs().simplify_full()

for ii in range(0,len(sus_func)):
    P22_value_22 = P22_value_22.subs(sus_func[ii])
    P22_value_33 = P22_value_33.subs(sus_func[ii])
    
P22_value_22 = P22_value_22.simplify_full()
P22_value_33 = P22_value_33.simplify_full()

P22_check_22_33 = P22_value_22 - P22_value_33

show(LatexExpr("P^{(2)}_2\lvert_{\\theta\\theta}=P^{(2)}_2\lvert_{\\phi\\phi}\\rightarrow"), bool(P22_check_22_33==0))

### <center> $G^{\prime\prime}_{00} = \kappa T^{\prime\prime}_{00}$ </center>

In [ ]:
ddEq_00 = (ddG[0,0] - kappa*ddT[0,0] == 0).simplify_full()

for ii in range(0,len(sus_legendre)):
    ddEq_00 = ddEq_00.subs(sus_legendre[ii])

ddEq_00 = ddEq_00.simplify_full()
ddEq_00 = (ddEq_00.subs(LP1==0) - ddEq_00.subs(LP1==0).subs(LP2==0))/LP2

E22_value_00 = solve(ddEq_00,E22)[0].rhs()
for ii in range(0,len(sus_func)):
    E22_value_00 = E22_value_00.subs(sus_func[ii])
E22_value_00 = E22_value_00.simplify_full()

ddE22_value_00 = diff(E22_value_00,r,r)
dE22_value_00 = diff(E22_value_00,r)
for ii in range(0,len(sus_func)):
    ddE22_value_00 = ddE22_value_00.subs(sus_func[ii])
    dE22_value_00 = dE22_value_00.subs(sus_func[ii])

ddG = ddG.subs(diff(E22,r,r)==ddE22_value_00).subs(diff(E22,r)==dE22_value_00).subs(E22==E22_value_00)
ddT = ddT.subs(diff(E22,r,r)==ddE22_value_00).subs(diff(E22,r)==dE22_value_00).subs(E22==E22_value_00)
sus_func.extend([diff(E22,r,r)==ddE22_value_00,
                 diff(E22,r)==dE22_value_00,
                 E22==E22_value_00])

### <center> $G^{\prime\prime}_{11} = \kappa T^{\prime\prime}_{11}$ </center>

In [ ]:
ddEq_11 = (ddG[1,1] - kappa*ddT[1,1] == 0).simplify_full()

for ii in range(0,len(sus_legendre)):
    ddEq_11 = ddEq_11.subs(sus_legendre[ii])

ddEq_11 = (ddEq_11.subs(LP1==0) - ddEq_11.subs(LP1==0).subs(LP2==0))/LP2

P22_value_11 = solve(ddEq_11,P22)[0].rhs()
for ii in range(0,len(sus_func)):
    P22_value_11 = P22_value_11.subs(sus_func[ii])
P22_value_11 = P22_value_11.simplify_full()

ddP22_value_11 = diff(P22_value_11,r,r)
dP22_value_11 = diff(P22_value_11,r)
for ii in range(0,len(sus_func)):
    ddP22_value_11 = ddP22_value_11.subs(sus_func[ii])
    dP22_value_11 = dP22_value_11.subs(sus_func[ii])

ddG = ddG.subs(diff(P22,r,r)==ddP22_value_11).subs(diff(P22,r)==dP22_value_11).subs(P22==P22_value_11)
ddT = ddT.subs(diff(P22,r,r)==ddP22_value_11).subs(diff(P22,r)==dP22_value_11).subs(P22==P22_value_11)
sus_func.extend([diff(P22,r,r)==ddP22_value_11,
                 diff(P22,r)==dP22_value_11,
                 P22==P22_value_11])

### <center> $G^{\prime\prime}_{12} = \kappa T^{\prime\prime}_{12}$ </center>

In [ ]:
ddEq_12 = ddG[1,2]-kappa*ddT[1,2] == 0

dk2_value = solve(ddEq_12, diff(k2,r))[0].rhs().simplify_full()

dk2_paper = -h2*diff(nu,r)-diff(h2,r)\
            +(1/r+diff(nu,r)/2)*(-2/3*r^3*j*diff(j,r)*(omega-Omega1)^2\
                                 +1/6*j^2*r^4*diff(omega,r)^2)

for ii in range(0,len(sus_func)):
    dk2_value = dk2_value.subs(sus_func[ii])
    dk2_paper = dk2_paper.subs(sus_func[ii])

dk2_value = dk2_value.simplify_full()
ddk2_value = diff(dk2_value,r)
for ii in range(0,len(sus_func)):
    ddk2_value = ddk2_value.subs(sus_func[ii])

ddG = ddG.subs(diff(k2,r,r)==ddk2_value).subs(diff(k2,r)==dk2_value)
ddT = ddT.subs(diff(k2,r,r)==ddk2_value).subs(diff(k2,r)==dk2_value)
sus_func.extend([diff(k2,r,r)==ddk2_value,
                 diff(k2,r)==dk2_value])

dk2_check = (dk2_value - dk2_paper).simplify_full()

show(LatexExpr("k_2' \\rightarrow"), bool(dk2_check==0))

### <center> $G^{\prime\prime}_{22} = \kappa T^{\prime\prime}_{22}$ </center>

In [ ]:
ddEq_22_00 = (ddG[2,2] - kappa*ddT[2,2] == 0).simplify_full()

for ii in range(0,len(sus_legendre)):
    ddEq_22_00 = ddEq_22_00.subs(sus_legendre[ii])

ddEq_22_00 = (ddEq_22_00.subs(LP1==0) - ddEq_22_00.subs(LP1==0).subs(LP2==0))/LP2
dh2_value = solve(ddEq_22_00, diff(h2,r))[0].rhs().simplify_full()

ddh2_value = diff(dh2_value,r)
for ii in range(0,len(sus_func)):
    ddh2_value = ddh2_value.subs(sus_func[ii])
ddh2_value = ddh2_value.simplify_full()

ddG = ddG.subs(diff(h2,r,r)==ddh2_value).subs(diff(h2,r)==dh2_value)
ddT = ddT.subs(diff(h2,r,r)==ddh2_value).subs(diff(h2,r)==dh2_value)
sus_func.extend([diff(h2,r,r)==ddh2_value,
                 diff(h2,r)==dh2_value])

dh2_paper = (-diff(nu,r)+r/((r-2*M)*diff(nu,r))*(8*pi*(E+P)-4*M/r^3))*h2\
            -4*(h2+k2)/(r*diff(nu,r)*(r-2*M))\
            +1/6*(1/2*r*diff(nu,r)-1/((r-2*M)*diff(nu,r)))*r^3*j^2*diff(omega,r)^2\
            -1/3*(1/2*r*diff(nu,r)+1/((r-2*M)*diff(nu,r)))*r^2*diff(j^2,r)*(omega-Omega1)^2

for ii in range(0,len(sus_func)):
    dh2_paper = dh2_paper.subs(sus_func[ii])

dh2_value = dh2_value.simplify_full()
dh2_check = dh2_value - dh2_paper

show(LatexExpr("h_{2}' \\rightarrow"), bool(dh2_check==0))

### <center> Ecuación 70 </center>

In [ ]:
E22_check = E22_value_00 - 4/3*diff(E,r)/diff(nu,r)*(3*h2+e^(-nu)*r^2*(omega-Omega1)^2)
for ii in range(0,len(sus_func)):
    E22_check = E22_check.subs(sus_func[ii])
E22_check = E22_check.simplify_full()

show(LatexExpr("\\text{Ecuación (70) }\\rightarrow"), bool(E22_check==0))

### <center> Ecuación 71 </center>

In [ ]:
P22_check = P22_value_11 + 2/3*(E+P)*(3*h2+e^(-nu)*r^2*(omega-Omega1)^2)
for ii in range(0,len(sus_func)):
    P22_check = P22_check.subs(sus_func[ii])
P22_check = P22_check.simplify_full()

show(LatexExpr("\\text{Ecuación (71) }\\rightarrow"), bool(P22_check==0))

### <center> Ecuación 73 </center>

In [ ]:
PT2 = P22/(2*(E+P))

eq_73 = PT2+h2+1/3*e^(-nu)*r^2*(omega-Omega1)^2
for ii in range(0,len(sus_func)):
    eq_73 = eq_73.subs(sus_func[ii])

show(LatexExpr("\\text{Ecuación (73) }\\rightarrow"), bool(eq_73==0))

## <center> Matching </center>

In [ ]:
var('a')

var('H0', latex_name="H_0")

var('s_k', latex_name="[k]")
var('s_h', latex_name="[h]")
var('s_v', latex_name="[v]")
var('s_dk', latex_name="[k^{\\prime}]")
var('s_dh', latex_name="[h^{\\prime}]")
var('s_dv', latex_name="[v^{\\prime}]")


var('s_h0', latex_name="[h_{0}]")
var('h0_n', latex_name="h_{0}^{-}")

var('s_v0', latex_name="[v_{0}]")
var('v0_n', latex_name="v_{0}^{-}")

var('s_dh0', latex_name="[h_{0}^{\\prime}]")
var('dh0_n', latex_name="h_{0}^{\\prime\, -}")

var('s_dv0', latex_name="[v_{0}^{\\prime}]")
var('dv0_n', latex_name="v_{0}^{\\prime\, -}")


var('s_k2', latex_name="[k_{2}]")
var('k2_a', latex_name="k_2(a)")

var('s_h2', latex_name="[h_{2}]")
var('h2_a', latex_name="h_2(a)")

var('s_v2', latex_name="[v_{2}]")
var('v2_n', latex_name="v_{2}^{-}")

var('s_dk2', latex_name="[k_{2}^{\\prime}]")
var('dk2_n', latex_name="k_{2}^{\\prime\, -}")

var('s_dh2', latex_name="[h_{2}^{\\prime}]")
var('dh2_n', latex_name="h_{2}^{\\prime\, -}")

var('s_dv2', latex_name="[v_{2}^{\\prime}]")
var('dv2_n', latex_name="v_{2}^{\\prime\, -}")


var('nu_a', latex_name="\\nu(a)")

var('dnu_a', latex_name="\\nu^{\\prime}(a)")

var('s_dlamb', latex_name="[\\lambda^{\\prime}]")
var('dlamb_n', latex_name="\\lambda^{\\prime}_{-}(a)")

var('s_ddnu', latex_name="[\\nu^{\\prime\\prime}]")
var('ddnu_n', latex_name="\\nu^{\\prime\\prime}_{-}(a)")

var('s_E', latex_name="[E]")
var('E_n', latex_name="E_{-}")


var('Q2', latex_name="\\hat{Q}_2")
var('Q20_a', latex_name="Q_{2(0)}(a)")
var('Q22_a', latex_name="Q_{2(2)}(a)")

var('domega_a', latex_name="\\omega^{\\prime}(a)")
var('omega_a', latex_name="\\omega(a)")

var('J')

var('PT0_a', latex_name="\\tilde{P}_{0}(a)")
var('P20_a', latex_name="P^{(2)}_{0}(a)")

M_a = a*(1-e^(nu_a))/2

In [ ]:
eq_s_k = s_k == 0
eq_s_h = s_h == 1/2*H0 + 1/2*a*dnu_a*s_k
eq_s_v = s_v == a*s_dk + 1/4*e^(nu_a/2)*s_dlamb*Q2 + 1/2*(a*dlamb_n+2)*s_k
eq_s_dh = s_dh == 1/2*a*dnu_a*s_dk + 1/4*e^(nu_a/2)*s_ddnu*Q2 + 1/2*(a*ddnu_n+dnu_a)*s_k

sus_s_descomp = [s_v==s_v0+s_v2*LP2,
                 s_dv==s_dv0+s_dv2*LP2,
                 s_k==s_k2*LP2,
                 s_dk==s_dk2*LP2,
                 s_h==s_h0+s_h2*LP2,
                 s_dh==s_dh0+s_dh2*LP2,
                 Q2==Q20_a+Q22_a*LP2]

for ii in range(0,len(sus_s_descomp)):
    eq_s_k = eq_s_k.subs(sus_s_descomp[ii])
    eq_s_h = eq_s_h.subs(sus_s_descomp[ii])
    eq_s_v = eq_s_v.subs(sus_s_descomp[ii])
    eq_s_dh = eq_s_dh.subs(sus_s_descomp[ii])

In [ ]:
sus_s_p = [M==M_a,
           lamb==-nu_a,
           diff(nu,r,r)==s_ddnu+ddnu_n,
           diff(nu,r)==dnu_a,
           nu==nu_a,
           diff(lamb,r)==s_dlamb+dlamb_n,
           diff(h2,r)==s_dh2+dh2_n,
           diff(omega,r)==domega_a,
           omega==omega_a,
           E==s_E,
           P==0]

sus_s_n = [M==M_a,
           lamb==-nu_a,
           diff(nu,r,r)==ddnu_n,
           diff(nu,r)==dnu_a,
           nu==nu_a,
           diff(lamb,r)==dlamb_n,
           diff(h2,r)==dh2_n,
           diff(omega,r)==domega_a,
           omega==omega_a,
           E==0,
           P==0]

sus_s = [s_ddnu==(1+a*dnu_a/2)*e^(-nu_a)*8*pi*s_E,
         s_dlamb==8*pi*a*e^(-nu_a)*s_E]

sus_s_func = [dnu_a==e^(-nu_a)*2*M_a/a^2,
              omega_a==2*J/a^3,
              r==a]

## <center>$\boxed{l=0}$</center>

## <center>[$h_0$]</center>

In [ ]:
s_h0_value = solve(eq_s_h, s_h0)[0].rhs()
s_h0_value = s_h0_value.subs(LP2==0)

s_h0_paper = H0/2
s_h0_check = s_h0_value - s_h0_paper

for ii in range(0,len(sus_s_func)):
    s_h0_value = s_h0_value.subs(sus_s_func[ii])
    s_h0_check = s_h0_check.subs(sus_s_func[ii])
    
s_h0_value = s_h0_value.simplify_full()

show(LatexExpr("[h_{0}]\\rightarrow"),bool(s_h0_check==0))

sus_s_p.extend([h0==s_h0+h0_n])
sus_s_n.extend([h0==h0_n])
sus_s.extend([s_h0==s_h0_value])

## <center>[$m_0$]</center>

In [ ]:
s_v0_value = solve(eq_s_v, s_v0)[0].rhs()
s_v0_value = s_v0_value.subs(LP2==0)

s_v0_paper = 2*pi*s_E*e^(-nu_a/2)*a*Q20_a
s_v0_check = s_v0_value - s_v0_paper

for ii in range(0,len(sus_s)):
    s_v0_value = s_v0_value.subs(sus_s[ii])
    s_v0_check = s_v0_check.subs(sus_s[ii])
    
for ii in range(0,len(sus_s_func)):
    s_v0_value = s_v0_value.subs(sus_s_func[ii])
    s_v0_check = s_v0_check.subs(sus_s_func[ii])

s_v0_value = s_v0_value.simplify_full()

show(LatexExpr("[v_{0}]\\rightarrow"),bool(s_v0_check==0))

sus_s_p.extend([v0==s_v0+v0_n])
sus_s_n.extend([v0==v0_n])
sus_s.extend([s_v0==s_v0_value])

## <center>[$h'_0$]</center>

In [ ]:
s_dh0_value = solve(eq_s_dh, s_dh0)[0].rhs()
s_dh0_value = s_dh0_value.subs(LP2==0)

s_dh0_paper = (a-M_a)/(a*(a-2*M_a))*s_v0
s_dh0_check = s_dh0_value - s_dh0_paper

for ii in range(len(sus_s)):
    s_dh0_value = s_dh0_value.subs(sus_s[ii])
    s_dh0_check = s_dh0_check.subs(sus_s[ii])
    
for ii in range(len(sus_s_func)):
    s_dh0_value = s_dh0_value.subs(sus_s_func[ii])
    s_dh0_check = s_dh0_check.subs(sus_s_func[ii])

s_dh0_value = s_dh0_value.simplify_full()

show(LatexExpr("[h'_{0}]\\rightarrow"),bool(s_dh0_check==0))

sus_s_p.extend([diff(h0,r)==s_dh0+dh0_n])
sus_s_n.extend([diff(h0,r)==dh0_n])
sus_s.extend([s_dh0==s_dh0_value])

## <center>$\boxed{l=2}$</center>

## <center> $[k_2]$

In [ ]:
s_k2_value = solve(eq_s_k, s_k2)[0].rhs()
s_k2_check = s_k2_value - 0

for ii in range(len(sus_s)):
    s_k2_value = s_k2_value.subs(sus_s[ii])

for ii in range(len(sus_s_func)):
    s_k2_value = s_k2_value.subs(sus_s_func[ii])

show(LatexExpr("[k_{2}]\\rightarrow"), bool(s_k2_check==0))

sus_s_p.extend([k2==k2_a])
sus_s_n.extend([k2==k2_a])
sus_s.extend([s_k2==s_k2_value])

## <center> $[h_2]$

In [ ]:
eq_s_h = eq_s_h - eq_s_h.subs(LP2==0)

s_h2_value = solve(eq_s_h, s_h2)[0].rhs()

s_h2_paper = 0
s_h2_check = s_h2_value - s_h2_paper

for ii in range(0,len(sus_s)):
    s_h2_value = s_h2_value.subs(sus_s[ii])
    s_h2_check = s_h2_check.subs(sus_s[ii])
    
for ii in range(0,len(sus_s_func)):
    s_h2_value = s_h2_value.subs(sus_s_func[ii])
    s_h2_check = s_h2_check.subs(sus_s_func[ii])

show(LatexExpr("[h_{2}]\\rightarrow"),bool(s_h2_check==0))

sus_s_p.extend([h2==h2_a])
sus_s_n.extend([h2==h2_a])
sus_s.extend([s_h2==s_h2_value])

## <center> $[h'_2]$ </center>

In [ ]:
dh2_p_value = dh2_value
dh2_n_value = dh2_value

for ii in range(len(sus_s_p)):
    dh2_p_value = dh2_p_value.subs(sus_s_p[ii])
    dh2_n_value = dh2_n_value.subs(sus_s_n[ii])

s_dh2_value_68 = dh2_p_value - dh2_n_value

s_dh2_paper = 4*pi*s_E*a^2/M_a*h2_a\
              +4/3*pi*s_E*a^2/M_a*e^(-2*nu_a)*((a-M_a)^2+M_a^2)*(2*J/a^3-Omega1)^2

s_dh2_check = s_dh2_value_68 - s_dh2_paper
for ii in range(len(sus_s)):
    s_dh2_value_68 = s_dh2_value_68.subs(sus_s[ii])
    s_dh2_check = s_dh2_check.subs(sus_s[ii])
    
for ii in range(len(sus_s_func)):
    s_dh2_value_68 = s_dh2_value_68.subs(sus_s_func[ii])
    s_dh2_check = s_dh2_check.subs(sus_s_func[ii])

show(LatexExpr("[h_{2}^{\\prime}]\\rightarrow"),bool(s_dh2_check==0))
sus_s_p.extend([diff(h2,r)==s_dh2+dh2_n])
sus_s_n.extend([diff(h2,r)==dh2_n])
sus_s.extend([s_dh2==s_dh2_value_68])

## <center> $Q_{2(2)}$ y $[k'_2]$ </center>

In [ ]:
s_dh2_value = (eq_s_dh - eq_s_dh.subs(LP2==0)).simplify_full()/LP2

for ii in range(len(sus_s)):
    s_dh2_value = s_dh2_value.subs(sus_s[ii])

for ii in range(len(sus_s_func)):
    s_dh2_value = s_dh2_value.subs(sus_s_func[ii])

s_dk2_value = solve(s_dh2_value, s_dk2)[0].rhs().simplify_full()

sus_s_p.extend([diff(k2,r)==s_dk2+dk2_n])
sus_s_n.extend([diff(k2,r)==dk2_n])
sus_s.extend([s_dk2==s_dk2_value])

In [ ]:
dk2_p_value = dk2_value
dk2_n_value = dk2_value

for ii in range(len(sus_s_p)):
    dk2_p_value = dk2_p_value.subs(sus_s_p[ii])
    dk2_n_value = dk2_n_value.subs(sus_s_n[ii])
    
for ii in range(len(sus_s)):
    dk2_p_value = dk2_p_value.subs(sus_s[ii])
    dk2_n_value = dk2_n_value.subs(sus_s[ii])

for ii in range(len(sus_s_func)):
    dk2_p_value = dk2_p_value.subs(sus_s_func[ii])
    dk2_n_value = dk2_n_value.subs(sus_s_func[ii])

s_dk2_value_67 = (dk2_p_value - dk2_n_value).simplify_full()
Q22_value_97 = s_dk2_value_67 - s_dk2_value == 0
Q22_value_97 = solve(Q22_value_97, Q22_a)[0].rhs()

Q22_paper = s_E*(h2_a-1/4*dnu_a*e^(nu_a/2)*Q22_a+1/3*a^2*e^(-nu_a)*(2*J/a^3-Omega1)^2)
Q22_paper = solve(Q22_paper, Q22_a)[0].rhs()

for ii in range(len(sus_s)):
    Q22_paper = Q22_paper.subs(sus_s[ii])
    Q22_paper = Q22_paper.subs(sus_s[ii])

for ii in range(len(sus_s_func)):
    Q22_paper = Q22_paper.subs(sus_s_func[ii])
    Q22_paper = Q22_paper.subs(sus_s_func[ii])

Q22_check = Q22_value_97 - Q22_paper

show(LatexExpr("Q_{(2)2}\\rightarrow"),bool(Q22_check ==0))
sus_s_func.extend([Q22_a==Q22_value_97])

In [ ]:
s_dk2_paper = -4*pi*s_E*a^2/M_a*h2_a\
              -4/3*pi*s_E*a^3/M_a*e^(-nu_a)*(a-2*M_a)*(2*J/a^3-Omega1)^2

s_dk2_check = s_dk2_value_67 - s_dk2_paper
for ii in range(len(sus_s)):
    s_dk2_value_67 = s_dk2_value_67.subs(sus_s[ii])
    s_dk2_check = s_dk2_check.subs(sus_s[ii])

for ii in range(len(sus_s_func)):
    s_dk2_value_67 = s_dk2_value_67.subs(sus_s_func[ii])
    s_dk2_check = s_dk2_check.subs(sus_s_func[ii])

show(LatexExpr("[k_{2}^{\\prime}]\\rightarrow"),bool(s_dk2_check==0))
sus_s_p.extend([diff(k2,r)==s_dk2+dk2_n])
sus_s_n.extend([diff(k2,r)==dk2_n])
sus_s.extend([s_dk2==s_dk2_value_67])

In [ ]:
s_v2_value = (eq_s_v - eq_s_v.subs(LP2==0)).simplify_full()/LP2

for ii in range(len(sus_s)):
    s_v2_value = s_v2_value.subs(sus_s[ii])

for ii in range(len(sus_s_func)):
    s_v2_value = s_v2_value.subs(sus_s_func[ii])

s_v2_value = solve(s_v2_value, s_v2)[0].rhs().simplify_full()
s_v2_paper = 8/3*pi*a^4*s_E*e^(-nu_a)*(2*J/a^3-Omega1)^2
s_v2_check = s_v2_value - s_v2_paper

show(LatexExpr("[v_{2}]\\rightarrow"),bool(s_v2_check==0))